In [1]:

from matplotlib import pyplot as plt
import seaborn as sns

import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

In [ ]:
times = ['time%s' % i for i in range(1, 11)]

train_df = pd.read_csv('train_sessions.csv',
                       index_col='session_id', parse_dates=times)

In [ ]:
train_df.head()

In [ ]:
train_df['year'] = train_df['time1'].apply(lambda ts : ts.year)

In [ ]:
train_df['month'] = train_df['time1'].apply(lambda ts : ts.month)

In [ ]:
train_df['week'] = train_df['time1'].apply(lambda ts : ts.week)

In [ ]:
train_df['day'] = train_df['time1'].apply(lambda ts : ts.day)

In [ ]:
train_df['dow'] = train_df['time1'].apply(lambda ts : ts.date().weekday())

In [ ]:
train_df['hour'] = train_df['time1'].apply(lambda ts : ts.hour)
train_df['sesslen'] = (train_df[times].max(axis = 1) - train_df[times].min(axis = 1)).apply(lambda ts: round(ts.seconds / 60))

In [ ]:
train_df['num_urls'] = train_df[times].count(axis = 1)

In [ ]:
train_df['active_days'] = (train_df['dow'].apply(lambda x : x in [0, 1, 3, 4]) ) * 1

In [ ]:
train_df['session_lenght'] = round((train_df[times].max(axis = 1) - train_df[times].min(axis = 1)).apply(lambda ts: ts.seconds)/60)

In [ ]:
train_df['active_hours'] = (train_df['hour'].apply(lambda x : x in [12, 13, 16, 17, 18]) ) * 1

In [ ]:
train_df['sub10'] = (train_df['num_urls'] < 10) * 1

In [ ]:
df = train_df

In [ ]:
sns.countplot(data = df, x = 'year')

In [ ]:
plt.figure(figsize=(20, 6))
plt.subplot(1, 3, 1)
sns.countplot(data = df, x = 'year')
plt.subplot(1, 3, 2)
plt.title('target = 1')
sns.countplot(data = df[df['target'] == 1], x = 'year')
plt.subplot(1, 3, 3)
sns.countplot(data = df[df['target'] == 0], x = 'year')

In [ ]:
plt.figure(figsize=(20, 6))
plt.subplot(1, 2, 1)
plt.title('target = 1')
sns.countplot(data = df[df['target'] == 1], x = 'hour')
plt.subplot(1, 2, 2)
plt.title('target = 0')
sns.countplot(data = df[df['target'] == 0], x = 'hour')


In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(20, 6))
plt.subplot(1, 2, 1)
plt.title('target = 1')
sns.countplot(data = df[df['target'] == 1], x = 'month')
plt.subplot(1, 2, 2)
plt.title('target = 0')
sns.countplot(data = df[df['target'] == 0], x = 'month')

In [ ]:
plt.figure(figsize=(20, 6))
plt.subplot(1, 2, 1)
plt.title('target = 1')
sns.countplot(data = df[df['target'] == 1], x = 'dow')
plt.subplot(1, 2, 2)
plt.title('target = 0')
sns.countplot(data = df[df['target'] == 0], x = 'dow')

In [ ]:
plt.figure(figsize=(20, 6))
plt.subplot(1, 2, 1)
plt.title('target = 1')
sns.countplot(data = df[df['sesslen'] == 1], x = 'dow')
plt.subplot(1, 2, 2)
plt.title('target = 0')
sns.countplot(data = df[df['sesslen'] == 0], x = 'dow')

In [ ]:
plt.figure(figsize=(20, 6))
plt.subplot(1, 2, 1)
plt.title('target = 1')
sns.countplot(data = df[df['target'] == 1], x = 'num_urls')
plt.subplot(1, 2, 2)
plt.title('target = 0')
sns.countplot(data = df[df['target'] == 0], x = 'num_urls')

In [ ]:
plt.figure(figsize=(20, 6))
plt.subplot(1, 2, 1)
plt.title('target = 1')
sns.countplot(data = df[df['target'] == 1], x = 'sub10')
plt.subplot(1, 2, 2)
plt.title('target = 0')
sns.countplot(data = df[df['target'] == 0], x = 'sub10')

In [ ]:
plt.figure(figsize=(20, 6))
plt.subplot(1, 2, 1)
plt.title('target = 1')
sns.countplot(data = df[df['target'] == 1], x = 'active_days')
plt.subplot(1, 2, 2)
plt.title('target = 0')
sns.countplot(data = df[df['target'] == 0], x = 'active_days')

In [ ]:
plt.figure(figsize=(20, 6))
plt.subplot(1, 2, 1)
plt.title('target = 1')
sns.countplot(data = df[df['target'] == 1], x = 'active_hours')
plt.subplot(1, 2, 2)
plt.title('target = 0')
sns.countplot(data = df[df['target'] == 0], x = 'active_hours')

In [ ]:
plt.figure(figsize=(20, 6))
plt.subplot(1, 2, 1)
plt.title('target = 1')
sns.countplot(data = df[df['target'] == 1], x = 'sesslen')
plt.subplot(1, 2, 2)
plt.title('target = 0')
sns.countplot(data = df[df['target'] == 0], x = 'sesslen')

In [ ]:
df.head()

In [ ]:
df.head()